# Hydrogen Atomic Orbitals

This document is a Jupyter notebook. If this is the first time you've worked with one, please take a moment to briefly read [these instructions](https://jupyter-notebook.readthedocs.io/en/stable/notebook.html#structure-of-a-notebook-document) explaining how to use it. In particular, make sure you understand the difference between a Markdown cell used for text/images/equations (such as the current one) and a code cell, which executes python code. When you do problem sets in this course, you will be expected to submit Jupyter Notebook files that have a mix of explanatory text and working code.

In this notebook, we will be visualizing the atomic orbitals of the hydrogen atom and learning about code vectorization along the way. Before starting, you should have reviewed the [Week 1 Background](https://kncrabtree.github.io/che155/weeks/1.html). The equation for a hydrogen atomic orbital is

$$ \psi_{n\ell m}(r,\theta,\phi) = \sqrt{ \left( \frac{2}{n} \right)^3 \frac{ (n-\ell-1)! }{2n(n+\ell)!} } e^{-r/2} r^\ell Y_\ell^m(\theta,\phi) L^{2\ell+1}_{n-\ell-1}(r) $$

## Getting started

Let's start by writing a python function that will calculate the value of the wavefunction at a point in space. To do so, the function requires $r$, $\theta$, $\phi$, $n$, $\ell$, and $m$. We also see that we need to evaluate factorials, a generalized Laguerre polynomial $L^{2\ell+1}_{n-\ell-1}(r)$, and a spherical harmonic function Y_\ell^m(\theta,\phi). In your earlier experience with python, you have probably encountered the [math.factorial](https://docs.python.org/3.7/library/math.html#math.factorial) function, but the standard python libraries do not have pre-existing implementations of $L^{2\ell+1}_{n-\ell-1}(r)$ and $Y_\ell^m(\theta,\phi)$. These functions are, however, available in SciPy, so you can save a lot of time using a premade implementation rather than writing your own. In addition, the functions available in SciPy are often based on well-known and studied implementations, and are much more likely to be efficient and bug-free than one you would come up with yourself. Where possible, always try to make use of SciPy and NumPy, as we will do in this course.

You will want to become comfortable reading the [NumPy/SciPy documentation](https://docs.scipy.org/doc/). This is where you will find lists of the functions that are available, explanations of how the functions work, and example code. On the documentation page, you will find links to documentation for different versions of SciPy and NumPy, so you need to select the version you are using. If you're not sure, it is easy to check. Execute the cell below to see what version you are using.

In [ ]:
import numpy as np
import scipy as sp

print(f'NumPy version {np.__version__}, SciPy version {sp.__version__}')

The functions we want are in the [`scipy.special` module](https://docs.scipy.org/doc/scipy/reference/tutorial/special.html), which contains [many other useful functions](https://docs.scipy.org/doc/scipy/reference/special.html#module-scipy.special) as well. In particular, we want:

- [`scipy.special.factorial`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.special.factorial.html#scipy.special.factorial)
- [`scipy.special.sph_harm`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.special.sph_harm.html#scipy.special.sph_harm)
- [`scipy.special.eval_genlaguerre`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.special.eval_genlaguerre.html#scipy.special.eval_genlaguerre)

With these in hand, we can write the definition of the hydrogen atomic orbital function. To save a bit of writing, we'll import the `scipy.special` module as `sps` to save some characters.

In [ ]:
import scipy.special as sps

def h_orbital(r,theta,phi,n,l,m):
    pf = ( (2./n)**3. * sps.factorial(n-l-1) / (2. * n * sps.factorial(n+l)) )**0.5
    return pf * np.exp(-r/2.) *r**l * sps.sph_harm(m,l,theta,phi) * sps.eval_genlaguerre(n-l-1,2*l+1,r)

## Important: Writing efficient code

At this point you may be wondering why we're using `sps.factorial` instead of `math.factorial`. What we'll soon see is that using the NumPy/SciPy versions of these functions, together with NumPy's [`ndarray` structure](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.html) allows for us to write *faster* and *more efficient* code.

To demonstrate this, we'll write some code using standard python data structures, and compare to the same code using a `numpy.ndarray`. We'll evaluate a $2p$ orbital with $m=0$ from 0 to 10 $a_0$ in steps of 0.1 $a_0$, holding $\theta=0$ and $\phi=0$.

In [ ]:
out = [] # create list to store output

for i in range(0,101): #loop from 0-100; divide by 10 to get desired value
    out.append(h_orbital(i/10.,0,0,2,1,0))

To do the same thing with an `ndarray`, we will first use the [`numpy.linspace`](https://numpy.org/doc/stable/reference/generated/numpy.linspace.html) function to create an array of $r$ values, and then we will call our `h_orbital` function with that array as the `r` argument. It may seem like this shouldn't work. After all, our function was written to expect a single value of `r`, and now we're giving it an array. However, all of NumPy and SciPy's functions are [universal functions](https://numpy.org/doc/stable/reference/ufuncs.html), and when they see an array as an argument, they *automatically* operate element-by-element on the array. Furthermore, because arrays are of fixed size and shape, the code is [vectorized](https://numpy.org/doc/stable/glossary.html#term-vectorization), and NumPy under the hood calls C-based implementations of loops that are much, much faster than python's `for` loops. This code accomplishes the same thing as before.

In [ ]:
r = np.linspace(0,10,101) #create an array of 101 points evenly spaced from 0 to 10 (inclusive)

out2 = h_orbital(r,0,0,2,1,0)

To test the efficiency of the code, we can use the ipython [`%timeit`](https://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-timeit) magic command, which executes a cell over and over for several seconds and computes the average amount of time it takes for the code to complete. To compare the efficiency, we'll just see how long it takes to compute the function, not worrying about capturing the output.

In [ ]:
%%timeit

for i in range(0,101): #loop from 0-100; divide by 10 to get desired value
    h_orbital(i/10.,0,0,2,1,0)

In [ ]:
%%timeit

h_orbital(r,0,0,2,1,0)

You can see that using the `ndarray` data structure to perform the vectorized computation is about 100 times faster in this test compared to a python `for` loop. Feel free to vary the sizes of the loops and compare!

This may not seem like such a big deal right now. Both implementations run effectively instantaneously. However, remember that the hydrogen orbitals are 3D functions, so we need to vary $\theta$ and $\phi$ in addition to $r$. Let's do that comparison. First, standard python.

In [ ]:
%%timeit

#Standard python -- this will take a while!

for r in range(0,101):
    for t in range(0,101):
        for p in range(0,101):
            h_orbital(r/10,t*np.pi/100,p*2*np.pi/100,2,1,0)

To do the same calculation with NumPy's `ndarray` structures, we can take advantage of [broadcasting](https://numpy.org/doc/stable/user/basics.broadcasting.html). Broadcasting allows for NumPy operations to automatically loop over arrays of different shapes. As explained in the documentation if we have a set of 100 x coordinates and a set of 100 y coordinates for a 2D function, and we want to evaluate at each x,y pair, we can axxomplish this by making `x` and `y` each 2D arrays with different shapes. `x` can be a 100x1 array (shape `(100,1)`), `y` can be a 1x100 array (shape `(1,100)`), and when we call `func(x,y)`, the output will be a 100x100 array (shape `(100,100)`).

In our case, we want to have a 3D grid of $(r,\theta,\phi)$ coordinates, each with 101 points. This means we can create a grid `rg` with shape `(101,1,1)`, `tg` with shape `(1,101,1)`, and `pg` with shape `(1,1,101)`. Then a call to `h_orbital(rg,tg,pg,2,1,0)` will return a 3D array with shape `(101,101,101)`, where the function has been evaluated at each point on the grid.

The function [`numpy.meshgrid`](https://numpy.org/doc/stable/reference/generated/numpy.meshgrid.html) can be used to conveniently construct a multidimensional grid like this from 1D arrays, such as the ones made by `numpy.linspace`. Read the documentation to understand why `indexing` and `sparse` are used below. You can also change them and see what happens.

In [ ]:
t = np.linspace(0,np.pi,101)
p = np.linspace(0,2.*np.pi,101)

print(f'r shape = {r.shape}, t shape = {t.shape}, p shape = {p.shape}')

rg,tg,pg = np.meshgrid(r,t,p,indexing='ij',sparse=True)

print(f'rg shape = {rg.shape}, tg shape = {tg.shape}, pg shape = {pg.shape}')

out3 = h_orbital(rg,tg,pg,2,1,0)

print (f'out3 shape = {out3.shape}, total number of elements: {out3.size}')

Now with the grids set up, we can do a timing comparison.

In [ ]:
%%timeit

h_orbital(rg,tg,pg,2,1,0)

In my trial this was about 3000 times faster than the python `for` loops! The take-home point of this section is that whenever possible, you should avoid using python's built-in `for` or `while` loops to loop through NumPy arrays. In fact, many other loop applications can also be avoided by clever use of `ndarray` and broadcasting, and in general this approach will be much faster. You can always test the performance of a code snippet with the `%timeit` magic function that we have been using; it's often a good way to identify performance bottlenecks and test ways to improve them.

## Introducing `matplotlib`

For data visualization, [`matplotlib`](https://matplotlib.org/) is one of the most commonly-used python packages, and it is the one we will use in this course. If this is the first time you are using `matplotlib`, take a moment to read through this basic [usage guide](https://matplotlib.org/tutorials/introductory/usage.html), which walks through the very basics of creating simple plots. Below we will begin generating plots, and the focus will mostly be on the hydrogen orbitals and the NumPy details, not on every matplotlib call. The usage guide will provide more explanation of many of the basics, and the API documentation for the [Axes](https://matplotlib.org/api/axes_api.html) and [Figure](https://matplotlib.org/api/_as_gen/matplotlib.figure.Figure.html) objects have many more details and options.

To begin, let's work with $s$ orbitals, which are spherically symmetric. This means that an $s$ orbital has the same value for every $\theta$ and $\phi$, and therefore the wavefunction only depends on $r$. While we could simply use 1D arrays for $r$ and $\psi$, we will still do the calculation in 3D so that our code is flexible. For this calculation, we'll use 1000 $r$ points, and 100 $\theta$ and $\pi$ points.

In [ ]:
#Specifying endpoint tells linspace whether to include the end value in the array
r = np.linspace(0,10.,1000,endpoint=False)
t = np.linspace(0,np.pi,100,endpoint=True)
p = np.linspace(0,2*np.pi,100,endpoint=False)

rg, tg, pg = np.meshgrid(r,t,p,indexing='ij',sparse=True)

psi_1s = h_orbital(rg,tg,pg,1,0,0)

print(f'psi_1s shape: {psi_1s.shape}')

As you read the `matplotlib` documentation, you will see that it references the "pyplot API" and the "object-oriented API." While both are perfectly valid, we will use the object-oriented API here. This is mostly to prevent errors that may happen when you use a notebook environment and execute cells in different orders. First we need to import the library and create a new set of `Figure` and `Axes` objects. The `Figure` is like the container for one or more plots, and the unfortunately-named `Axes` object (not to be confused with an `Axis` object!) is one of the plots. We can create the `Figure` and `Axes` with another unfortunately-named command: [`pyplot.subplots`](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.subplots.html?highlight=subplots#matplotlib.pyplot.subplots). Although that function can create a figure with multiple subplots, when called without arguments, it actualy produces a figure with only one plot, which is what we want in this case.

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()

We'll start by making a 1D visualization of the $1s$ orbital as a function of $r$ at a single value of $\theta$ and $\phi$. To do this, we will take [slices](https://numpy.org/doc/stable/reference/arrays.indexing.html#basic-slicing-and-indexing) of the `psi_1s` array. We're also using the [`np.real`](https://numpy.org/doc/stable/reference/generated/numpy.real.html) function to take only the real part of the wavefunction. The $1s$ orbital is entirely real anyways, so this does not discard any information; later we will deal with other wavefunctions that are complex.

In [ ]:
ax.cla() #clear the axes object in case things have already been plotted on it
ax.plot(rg[:,0,0],np.real(psi_1s[:,0,0]),label='$\psi_{1s}$')
ax.set_xlabel('$r$ ($a_0$)')
ax.set_ylabel('$\psi$')
fig #doing this will make sure the updated figure is plotted again.

A plot can have more than one curve on it. Here we'll calculate the $2s$ orbital, plot it, and add a legend.

In [ ]:
psi_2s = h_orbital(rg,tg,pg,2,0,0)

In [ ]:
ax.plot(rg[:,0,0],np.real(psi_2s[:,0,0]),label='$\psi_{2s}$')
ax.legend()
fig

To better see the node structure, we can add a dashed horizontal black line.

In [ ]:
ax.plot([0,10],[0,0],'k--')
fig